Prepare input data to be ready to train a TTS model. 

# Imports

In [1]:
import IPython
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import glob
import os
import shutil
from scipy.io import wavfile


In [3]:
# Style overrides
from IPython.core.display import HTML
css = open("../../QA/overrides.css", "r").readlines()
css = "".join(css)
css = f"<style>{css}</style>"
HTML(css)

In [4]:
import logging

# Config

In [5]:
# Logging 
logging.basicConfig(
     #filename='DockProc.log',
     level=logging.DEBUG, 
     format= '[%(asctime)s] {%(lineno)d} %(levelname)s - %(message)s',
     datefmt='%H:%M:%S'
 )
logger = logging.getLogger("VoiceBox")
logger.setLevel("DEBUG")

# Resources
device = "cpu"

In [6]:
def play_audio(wav, sr=22050):
    IPython.display.display(IPython.display.Audio(wav, rate=sr, autoplay=False))


In [7]:

def expose(prefix:str=""):
    import torch
    print (f"{prefix}{torch.__version__ = }")
    import transformers
    print (f"{prefix}{transformers.__version__ = }")
    import nltk
    print (f"{prefix}{nltk.__version__ = }")
    import speechbrain
    print (f"{prefix}{speechbrain.__version__ = }")
    import TTS
    print (f"{prefix}{TTS.__version__ = }")
    import datasets
    print (f"{prefix}{datasets.__version__ = }")
expose()


torch.__version__ = '2.0.1+cu118'
transformers.__version__ = '4.35.0.dev0'


[21:41:41] {75} DEBUG - Failed to initialize ffmpeg bindings
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torchaudio/_extension/utils.py", line 85, in _init_ffmpeg
    _load_lib("libtorchaudio_ffmpeg")
  File "/opt/conda/lib/python3.10/site-packages/torchaudio/_extension/utils.py", line 61, in _load_lib
    torch.ops.load_library(path)
  File "/opt/conda/lib/python3.10/site-packages/torch/_ops.py", line 643, in load_library
    ctypes.CDLL(path)
  File "/opt/conda/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: /usr/lib/x86_64-linux-gnu/libgobject-2.0.so.0: undefined symbol: ffi_type_uint32, version LIBFFI_BASE_7.0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torchaudio/_extension/__init__.py", line 67, in <module>
    _init_ffmpeg()
  File "/opt/conda/lib/python3.10/site-packag

nltk.__version__ = '3.8.1'
speechbrain.__version__ = '0.5.14'
TTS.__version__ = '0.19.1'


[21:41:41] {148} INFO - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[21:41:41] {160} INFO - NumExpr defaulting to 8 threads.


datasets.__version__ = '2.14.5'


# Clean transcript

In [3]:
data_folder = "../data"

In [11]:
full_transcript_path = f"{data_folder}/full_lee_test_44//transcript.txt"

In [ ]:
transcript = ""
with open(full_transcript_path, "r") as full_transcript:
        transcript = full_transcript.readlines()
        full_transcript.close()
        
for index in range(len(transcript)):
    text = transcript[index]
    text = text.replace(".wav","")
    
    #text = text.replace("\n","")
    text = text.replace("f******","fucking")
    text = text.replace("f***","fuck")
    text = text.replace("fuck**","fucked")
    text = text.replace("F***","Fuck")
    text = text.replace("b****","bitch")
    text = text.replace("b*******","bitchass")
    text = text.replace("bitch***","bitchass")
    
    text = text.replace("c*********", "cocksucker")
    text = text.replace("ass****", "asshole")
    
    
    text = text.replace("s***","shit")
    text = text.replace("p***","puss")
    text = text.replace("a**","ass")
    text = text.replace("\\","")
    text = text.replace("/"," ")
    text = text.replace("%","")
    text = text.replace("-"," ")
    text = text.replace("*"," ")
    if "*" in text:
        print (text)
    
    
    replacements = [
        ('à', 'a'),
        ('ç', 'c'),
        ('è', 'e'),
        ('ë', 'e'),
        ('í', 'i'),
        ('ï', 'i'),
        ('ö', 'o'),
        ('ü', 'u'),
    ]
    
    for src, dst in replacements:
        text = text.replace(src, dst)
        
    
    transcript[index] = text

# Set output path for transcript
full_transcript_path = f"{data_folder}/full_{dataset_prefix}_test_44/transcript.txt"

with open(full_transcript_path, "w") as full_transcript:
        #np.save(full_transcript_path, np.array(transcript))
        #json.dump(transcript, full_transcript)
        full_transcript.writelines(transcript)
        full_transcript.close()
    

In [5]:
# Verify written transcript
transcript = ""
with open(full_transcript_path, "r") as full_transcript:
        transcript = full_transcript.readlines()
        full_transcript.close()
len(transcript)

37648

In [6]:
transcript

['lee_test_44_h32_0|Ice front\n',
 "lee_test_44_h32_1|Right here it's time for mail call\n",
 'lee_test_44_h32_2|Welcome to mail call\n',
 'lee_test_44_h32_3|This is the way we\n',
 "lee_test_44_h32_4|What's the train back when I was the grunt\n",
 'lee_test_44_h32_5|Down and\n',
 'lee_test_44_h32_6|Play just the way we\n',
 "lee_test_44_h32_7|Nowadays there's still a lot of good old fashioned\n",
 "lee_test_44_h32_8|But they're also\n",
 'lee_test_44_h32_9|Listen to mind blowing electronics\n',
 'lee_test_44_h32_10|Just to get our troops battle ready\n',
 'lee_test_44_h32_11|I got an email\n',
 'lee_test_44_h32_12|Can Twin Lakes Wisconsin\n',
 'lee_test_44_h32_13|I need to show you guys how\n',
 'lee_test_44_h32_14|Open ship training is handled today\n',
 'lee_test_44_h32_15|US Marines still get down in the dirt for weapons training it outdoor\n',
 'lee_test_44_h32_16|Change is like this one at Camp Pendleton California\n',
 'lee_test_44_h32_17|But when they want to step things up a n

# Evaluate examples

In [12]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
model.config.forced_decoder_ids = None

[21:42:10] {456} DEBUG - https://huggingface.co:443 "HEAD /openai/whisper-large-v2/resolve/main/preprocessor_config.json HTTP/1.1" 200 0
[21:42:10] {456} DEBUG - https://huggingface.co:443 "HEAD /openai/whisper-large-v2/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
[21:42:10] {456} DEBUG - https://huggingface.co:443 "HEAD /openai/whisper-large-v2/resolve/main/config.json HTTP/1.1" 200 0
[21:42:10] {456} DEBUG - https://huggingface.co:443 "HEAD /openai/whisper-large-v2/resolve/main/config.json HTTP/1.1" 200 0
[21:42:17] {456} DEBUG - https://huggingface.co:443 "HEAD /openai/whisper-large-v2/resolve/main/generation_config.json HTTP/1.1" 200 0


In [10]:
data_folder = "../data/full_lee_44"
wav_folder = "../data/full_lee_22/wavs"
transcript = ""

with open(full_transcript_path, "r") as full_transcript:
        transcript = full_transcript.readlines()
        full_transcript.close()



NameError: name 'full_transcript_path' is not defined

In [ ]:
# Open the wav file
from scipy.io import wavfile
samplerate, wav = wavfile.read(input_file)

In [ ]:
samplerate

In [ ]:
index = 0

In [ ]:
parts = transcript[index].split("|")

In [ ]:
filename = f"{parts[0]}"

In [ ]:
filename

In [ ]:
filepath = f"{wav_folder}/{filename}"
samplerate, wav = wavfile.read(filepath)

In [ ]:
play_audio(wav, samplerate)

In [ ]:
wav = np.array(wav, dtype=np.float32)

In [ ]:
input_features = processor(wav, sampling_rate=samplerate, return_tensors="pt").input_features 

In [ ]:
input_features[0][0]

In [ ]:
input_features.shape

In [ ]:
# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

In [ ]:
transcription

In [ ]:
play_audio(wav, samplerate)